In [10]:
import cv2
import dlib
import numpy as np
import pygame
from scipy.spatial import distance as dist
import requests
import os

# تحقق إذا كان الملف موجود، لو مش موجود هيحمل
SHAPE_PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"
SHAPE_PREDICTOR_URL = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"

if not os.path.exists(SHAPE_PREDICTOR_PATH):
    print("جاري تحميل ملف shape_predictor_68_face_landmarks.dat...")
    response = requests.get(SHAPE_PREDICTOR_URL)
    with open("shape_predictor_68_face_landmarks.dat.bz2", "wb") as f:
        f.write(response.content)
    
    import bz2
    with bz2.open("shape_predictor_68_face_landmarks.dat.bz2", "rb") as f_in:
        with open(SHAPE_PREDICTOR_PATH, "wb") as f_out:
            f_out.write(f_in.read())
    
    os.remove("shape_predictor_68_face_landmarks.dat.bz2")
    print("تم تحميل الملف بنجاح!")

# تحميل موديل dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(SHAPE_PREDICTOR_PATH)

# دالة لحساب نسبة إغلاق العين (EAR) بدقة أعلى
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])  # الطرف العلوي للعين
    B = dist.euclidean(eye[2], eye[4])  # الطرف السفلي للعين
    C = dist.euclidean(eye[0], eye[3])  # العرض الأفقي للعين
    ear = (A + B) / (2.0 * C)
    return ear

# مسار الصوت الموجود على جهازك
ALARM_PATH = "adel.mp3"  # استبدل المسار بملف صوتي موجود
try:
    pygame.mixer.init()
    alarm_sound = pygame.mixer.Sound(ALARM_PATH)
    print("تم تحميل الصوت بنجاح!")
except Exception as e:
    print(f"فشل تحميل الصوت: {e}")

is_alarm_playing = False

# فتح الكاميرا
cap = cv2.VideoCapture(1)

# عتبة لتحديد النعاس (EAR)
EAR_THRESHOLD = 0.14  # قيمة متوازنة لزيادة الدقة
EYE_CLOSED_FRAMES = 4  # عدد فريمات للتأكد من النعاس
EYE_OPEN_THRESHOLD = 0.3  # عتبة للتأكد إن العين مفتوحة
counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    sleeping = False
    eyes_closed = False

    for face in faces:
        landmarks = predictor(gray, face)
        
        # استخراج نقاط العين (يمين وشمال)
        left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
        right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

        # حساب EAR لكل عين
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # رسم نقاط العين على الشاشة
        for (x, y) in left_eye + right_eye:
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        # تحديد إذا كانت العين مغلقة
        if ear < EAR_THRESHOLD:
            counter += 1
            eyes_closed = True
            if counter >= EYE_CLOSED_FRAMES:
                sleeping = True
                cv2.putText(frame, "WARNING: sleeping driver! (Eyes Closed)", (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        else:
            # التأكد إن العين مفتوحة بالكامل
            if ear > EYE_OPEN_THRESHOLD:
                counter = 0  # إعادة العداد لو العين مفتوحة
            eyes_closed = False
            sleeping = False

        # عرض حالة العين
        eye_status = "Closed" if eyes_closed else "Open"
        cv2.putText(frame, f"Eyes: {eye_status}", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # عرض حالة الشخص
        status = "Sleeping" if sleeping else "Awake"
        cv2.putText(frame, f"Status: {status}", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # عرض قيمة EAR
        cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # تشغيل/إيقاف الصوت
    if sleeping and not is_alarm_playing:
        try:
            alarm_sound.play(-1)
            is_alarm_playing = True
            print("تشغيل الصوت...")
        except Exception as e:
            print(f"فشل تشغيل الصوت: {e}")
    elif not sleeping and is_alarm_playing:
        try:
            alarm_sound.stop()
            is_alarm_playing = False
            print("إيقاف الصوت...")
        except Exception as e:
            print(f"فشل إيقاف الصوت: {e}")

    cv2.imshow("Driver Monitoring", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()

تم تحميل الصوت بنجاح!
تشغيل الصوت...
إيقاف الصوت...
